# Export Bathymetry

Exports bathymetry tiles from a geometry of interest to GEE assets and GCS.
TODO: create logic in outside of notebook

In [6]:
from pathlib import Path
import sys
import time

import ee
import geemap
import geetools
import geojson

sys.path.append(str(Path.cwd().parent.parent))

from eobathymetry import bathymetry, tiler

## Create tiles based on input geometry

In [2]:
Map = geemap.Map(center=(52.97, 4.74), zoom=10)

"""
var tiler = require('users/gena/packages:tiler')

function addTileBounds(zoom){
  var tiles = tiler.getTilesForGeometry(geometry, zoom)
  
  Map.addLayer(tiles.style({ width: Math.max(1, 10 - zoom), fillColor: '00000022' }), {}, 'tiles ' + zoom)
}

[5, 6, 7, 8, 9].map(addTileBounds)
print(texelAoi)
"""

aoi_json = """{
  "type": "Polygon",
  "coordinates": [
    [
      [
        4.726555204480136,
        52.79894952106581
      ],
      [
        5.382988309948886,
        53.2615577684405
      ],
      [
        5.226433134167636,
        53.48931215536743
      ],
      [
        4.770500516980136,
        53.41898585234949
      ],
      [
        4.270622587292636,
        52.91018589685636
      ],
      [
        4.726555204480136,
        52.79894952106581
      ]
    ]
  ]
}"""
bounds = ee.Geometry(geojson.loads(aoi_json))
zoom_levels = [8, 9, 10, 11, 12]

## Plot tiles for quality control

In [3]:
def add_tile_bounds(zoom):
    tiles = tiler.get_tiles_for_geometry(bounds, zoom)
    Map.addLayer(tiles.style(width=max(1, 10 - zoom), fillColor= "00000022"), {}, "tiles " + str(zoom))

In [4]:
list(map(add_tile_bounds, [8, 9, 10, 11, 12]))
Map

Map(center=[52.97, 4.74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

## Export to earth engine assets
Create tiles to export. Then export based tiles and given zoom levels.

In [12]:
from typing import List
from dateutil.parser import parse

def export_tile_bathymetry(tile: ee.Feature, start: str, stop: str, preview: bool=False):
    bounds: ee.Geometry = tile.geometry().bounds(1)
    scale: ee.Number = tiler.zoom_to_scale(tile.get("zoom"))
    sdb: Bathymetry = bathymetry.Bathymetry()
    tile.set(ee.String("z").cat(tile.get("zoom")).cat("_x").cat(tile.get("tx")).cat("_y").cat(tile.get("ty")))
    
    image: ee.Image = sdb.compute_inverse_depth(
                bounds=bounds,
                start=start,
                stop=stop,
                filter_masked=True,
                scale=tiler.zoom_to_scale(zoom)
            )
    image.set(
        "system:time_start", ee.Date(start).millis(),
        "system:time_stop", ee.Date(stop).millis(),
        "zoom", zoom,
        "tx", tile.get("tx"),
        "ty", tile.get("ty")
    )
    
    buffered_bounds: ee.Geometry = tile.geometry().buffer(scale).bounds()
    if preview:
        Map.addLayer(image.clip(bounds), {}, "preview SDB")
        return
    
    def export_tiled_feature(name: ee.String):
        file_name: str = f"{name}_{start}_{stop}"
        user_name = ee.data.getAssetRoots()[0]["id"].split("/")[-1]
        asset_id: str = f"users/{user_name}/eo-bathymetry/{file_name}"
        ee.batch.Export.image.toAsset(
            image=image,
            description=file_name,
            assetId=asset_id,
            region=bounds,
            scale=scale,
            crs="EPSG:3857",
            maxPixels=1e10
        )
    
    tile.get("name").evaluate(export_tiled_feature)

def export_tiles_to_assets(
    geometry: ee.Geometry,
    sdb: bathymetry.Bathymetry,
    zoom_levels: List[int],
    start: str,
    stop: str,
    step_months: int = 3,
    window_years: int = 2,
    preview: bool = False
):
    user_name = ee.data.getAssetRoots()[0]["id"].split("/")[-1]
    asset_id: str = f"users/{user_name}/eo-bathymetry"
    try:
        ee.data.deleteAsset(asset_id)
    except ee.EEException as e:
        if f"Asset \"projects/earthengine-legacy/assets/{asset_id}\" not found." not in str(e):
            raise e
        else:
            print(f"Asset {asset_id} not found")
    
    ee.data.create_assets(asset_ids=["/".join(asset_id.split("/")[:-1])], asset_type="Folder", mk_parents=True)
    
    def create_year_window(month: ee.Number) -> ee.Dictionary:
        t: ee.Date = ee.Date.fromYMD(year, month, 1)
        d_format: str = "YYYY-MM-dd"
        return ee.Dictionary({"start": t.format(d_format), "stop": t.advance(window_years, 'year').format(d_format)})
        
    dates: ee.List = ee.List.sequence(parse(start).year, parse(stop).year).map(
        lambda year: ee.List.sequence(1, 12, step_months).map(create_year_window)
    ).flatten()
    
    zooms: ee.List = ee.List(zoom_levels)
    zooms \
        .map(lambda zoom_l: tiler.get_tiles_for_geometry(geometry, zoom)) \
#         .flatten() \
#         .map(lambda tile: export_tile_bathymetry(tile, start, stop, preview))

In [13]:
export_tiles_to_assets(bounds, bathymetry.Bathymetry(), [10, 11], "01-01-2020", "01-01-2021", True)
Map

EEException: An internal error has occurred (request: fe97ee67-512e-4987-861b-83824fd86673)